# SPR 2026 - Word2Vec + Max Pooling

**Comparação de estratégias de agregação**

Max Pooling captura as features mais salientes de cada dimensão.
Também testamos Mean + Max concatenados.

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Gensim já está pré-instalado no Kaggle
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
import re
import warnings
warnings.filterwarnings('ignore')

SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# Preprocessamento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

print(f'Exemplo: {train["tokens"].iloc[0][:10]}')

In [ ]:
# Treinar Word2Vec
all_texts = train['tokens'].tolist() + test['tokens'].tolist()

w2v = Word2Vec(
    sentences=all_texts,
    vector_size=EMBEDDING_DIM,
    window=5,
    min_count=2,
    workers=4,
    epochs=10,
    seed=SEED
)

print(f'Vocabulário: {len(w2v.wv)} palavras')

In [ ]:
# Diferentes estratégias de agregação
def text_to_embedding_mean(tokens, model, dim):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

def text_to_embedding_max(tokens, model, dim):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.max(vectors, axis=0)

def text_to_embedding_mean_max(tokens, model, dim):
    """Concatena Mean e Max pooling -> 2x dimensão"""
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim * 2)
    mean_vec = np.mean(vectors, axis=0)
    max_vec = np.max(vectors, axis=0)
    return np.concatenate([mean_vec, max_vec])

In [ ]:
# Usar Mean + Max (200 dimensões)
POOLING = 'mean_max'  # Options: 'mean', 'max', 'mean_max'

if POOLING == 'mean':
    X_train = np.array([text_to_embedding_mean(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
    X_test = np.array([text_to_embedding_mean(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
elif POOLING == 'max':
    X_train = np.array([text_to_embedding_max(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
    X_test = np.array([text_to_embedding_max(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
else:  # mean_max
    X_train = np.array([text_to_embedding_mean_max(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
    X_test = np.array([text_to_embedding_mean_max(t, w2v, EMBEDDING_DIM) for t in test['tokens']])

y_train = train['target'].values

print(f'Pooling: {POOLING}')
print(f'X_train shape: {X_train.shape}')

In [ ]:
# Treinar Logistic Regression
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)

model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())